Case Studies

Project: 1

Group: 3

Group Members:
 - Muhammad Raafey Tariq (231806)
 - Farrukh Ahmed (230614)
 - Amirreza Khamehchin Khiabani (230891)
 - Aymane Hachcham (236392)


Requirements:
 - numpy==1.24.2
 - matplotlib==3.7.1
 - seaborn==0.12.2
 - pandas==2.0.0
 - openpyxl==3.1.2

Installation Commands (One-time only)
 - pip install pandas==2.0.0
 - pip install numpy==1.24.2
 - pip install seaborn==0.12.2
 - pip install matplotlib==3.7.1
 - pip install openpyxl==3.1.2

Imports and Libraries

In [23]:
import pandas as pd
import numpy as np
import itertools
import pprint
import random

# used for the graphs
import seaborn as sns

import os
sns.set(font_scale = 1.2)

# used for plotting
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from helper_functions import *


import matplotlib

# setting font to 'Times New Roman'
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams.update({'font.size': 16})
%matplotlib inline

Global Variables and Constants

Importing Data

In [24]:
data_df = read_data()

c:\Users\Raafe\AppData\Local\Programs\Python\Python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Reformating Columns to Correct Data Types and dropping nans

In [25]:
# variables that are dropped 
to_filter = ["id", "zeit", "postleitzahl", "gemeinde", "bezirk", "geburtsjahr", "schaetzwert_bp_sys", "schaetzwert_by_dia"]
data_df, cat_feat_list, num_feat_list = format_variables(data_df, to_filter=to_filter, drop_values=True)

# one hot encoding cat variables to prep data for Decision Tree
# ordinal variables and nominal are treated the same in trees, but need to be careful in Lin models

encoded_data_df = encode_data(data_df, cat_feat_list, num_feat_list)
encoded_train_set, encoded_test_set = train_test_split(encoded_data_df, test_size=0.3)

In [26]:
print("Size of training data: ", len(encoded_train_set))
print("Size of testing data: ", len(encoded_test_set))
print("Features used: ", data_df.columns)

Size of training data:  10381
Size of testing data:  4450
Features used:  Index(['bundesland', 'befinden', 'geschlecht', 'raucher', 'blutzucker_bekannt',
       'cholesterin_bekannt', 'in_behandlung', 'messwert_bp_sys',
       'messwert_bp_dia', 'month', 'hour', 'day', 'age'],
      dtype='object')


https://scikit-learn.org/stable/modules/tree.html#tree

 - scikit-learn uses an optimized version of the CART algorithm, does not support categorical variables
 - BIC cannot be computed as it depends on likelihood, cannot compute that for RegressionTree as it does not assume a conditional dist of data

In [27]:
# using self evaluated sys bp for analysis
target = "messwert_bp_sys"

# splitting targets from predictors
X_train, Y_train = separate_target(encoded_train_set, target)
X_test, Y_test = separate_target(encoded_test_set, target)

In [28]:
# fitting base model for DecisonTreeRegressor using all available features and default parameters

train_results_tree_base, test_results_tree_base, model_tree_base = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressor", params={"criterion" : "squared_error"})

In [29]:
# fitting base model for RandomForestRegressor using all available features and default parameters

train_results_rf_base, test_results_rf_base, model_rf_base = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressorRandomForest", {"criterion" : "squared_error",
                                                                                                                    "n_estimators": 100})

In [30]:
# finding the best set of parameters to use by finetuning RegTree using CV, fine-tuning is done on whole dataset

parameters= {"splitter":["best","random"],
            "max_depth" : list(np.arange(1, 25, 5, dtype=int)),
           "min_samples_leaf":list(np.arange(1, 100, 5, dtype=int)),
           "min_weight_fraction_leaf":list(np.arange(0, 1, 1.0, dtype=float)),
           "max_features":list(np.arange(1, len(X_train.columns), 1, dtype=int))
           }

model = DecisionTreeRegressor()
X_train_full, Y_train_full = separate_target(encoded_data_df, target)
tuning_model = GridSearchCV(model, param_grid=parameters, scoring='neg_mean_squared_error',cv=10,verbose=0)
tuning_model.fit(X_train_full, Y_train_full)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [1, 6, 11, 16, 21],
                         'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22],
                         'min_samples_leaf': [1, 6, 11, 16, 21, 26, 31, 36, 41,
                                              46, 51, 56, 61, 66, 71, 76, 81,
                                              86, 91, 96],
                         'min_weight_fraction_leaf': [0.0],
                         'splitter': ['best', 'random']},
             scoring='neg_mean_squared_error')

In [31]:
tuning_model.best_params_

{'max_depth': 16,
 'max_features': 22,
 'min_samples_leaf': 91,
 'min_weight_fraction_leaf': 0.0,
 'splitter': 'best'}

In [32]:
# fitting fine_tuned model for DecisonTreeRegressor using all available features and fine_tuned parameters
model_params = tuning_model.best_params_.copy()
model_params["criterion"] = "squared_error"

train_results_tree_fine, test_results_tree_fine, model_tree_fine = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressor",
                                                                              model_params)

In [33]:
# fitting fine_tuned model for RandomForestRegressor using all available features and fine_tuned parameters
model_params = tuning_model.best_params_.copy()
model_params["criterion"] = "squared_error"
del model_params["splitter"]
model_params["n_estimators"] = 100

train_results_rf_fine, test_results_rf_fine, model_rf_fine = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressorRandomForest",
                                                                              model_params)

In [34]:
train_result_list = [train_results_tree_base, train_results_tree_fine, train_results_rf_base, train_results_rf_fine]

test_result_list = [test_results_tree_base, test_results_tree_fine, test_results_rf_base, test_results_rf_fine]

model_names = ["Tree (Base)", "Tree (Fine-tuned)",
               "RF (Base)", "RF (Fine-tuned)"]

tab = tabularize_model_metrics(train_result_list, test_result_list, model_names)
round(tab, 3)

,Model,Train Mean Sq Error,Test Mean Sq Error,Train R2,Test R2,Train Adjusted R2,Test Adjusted R2
0,Tree (Base),0.066,356.951,1.000,0.026,1.000,0.021
1,Tree (Fine-tuned),169.802,179.338,0.543,0.510,0.542,0.508
2,RF (Base),25.231,177.910,0.932,0.514,0.932,0.512
3,RF (Fine-tuned),167.988,172.791,0.548,0.528,0.547,0.526


In [35]:
print(round(tab, 2).to_latex())

\begin{tabular}{llrrrrrr}
\toprule
 & Model & Train Mean Sq Error & Test Mean Sq Error & Train R2 & Test R2 & Train Adjusted R2 & Test Adjusted R2 \\
\midrule
0 & Tree (Base) & 0.070000 & 356.950000 & 1.000000 & 0.030000 & 1.000000 & 0.020000 \\
1 & Tree (Fine-tuned) & 169.800000 & 179.340000 & 0.540000 & 0.510000 & 0.540000 & 0.510000 \\
2 & RF (Base) & 25.230000 & 177.910000 & 0.930000 & 0.510000 & 0.930000 & 0.510000 \\
3 & RF (Fine-tuned) & 167.990000 & 172.790000 & 0.550000 & 0.530000 & 0.550000 & 0.530000 \\
\bottomrule
\end{tabular}



In [36]:
# using self evaluated dia bp for analysis
target = "messwert_bp_dia"

# splitting targets from predictors
X_train, Y_train = separate_target(encoded_train_set, target)
X_test, Y_test = separate_target(encoded_test_set, target)

In [37]:
# fitting base model for DecisonTreeRegressor using all available features and default parameters

train_results_tree_base, test_results_tree_base, model_tree_base = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressor", {"criterion" : "squared_error"})

In [38]:
# fitting base model for RandomForestRegressor using all available features and default parameters

train_results_rf_base, test_results_rf_base, model_rf_base = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressorRandomForest", {"criterion" : "squared_error",
                                                                                                                    "n_estimators": 100})

In [39]:
# finding the best set of parameters to use by finetuning RegTree using CV, fine-tuning is done on whole dataset

parameters= {"splitter":["best","random"],
            "max_depth" : list(np.arange(1, 25, 5, dtype=int)),
           "min_samples_leaf":list(np.arange(1, 100, 5, dtype=int)),
           "min_weight_fraction_leaf":list(np.arange(0, 1, 1.0, dtype=float)),
           "max_features":list(np.arange(1, len(X_train.columns), 1, dtype=int))
           }

model = DecisionTreeRegressor()
X_train_full, Y_train_full = separate_target(encoded_data_df, target)
tuning_model = GridSearchCV(model, param_grid=parameters, scoring='neg_mean_squared_error',cv=10,verbose=0)
tuning_model.fit(X_train_full, Y_train_full)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [1, 6, 11, 16, 21],
                         'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22],
                         'min_samples_leaf': [1, 6, 11, 16, 21, 26, 31, 36, 41,
                                              46, 51, 56, 61, 66, 71, 76, 81,
                                              86, 91, 96],
                         'min_weight_fraction_leaf': [0.0],
                         'splitter': ['best', 'random']},
             scoring='neg_mean_squared_error')

In [40]:
tuning_model.best_params_

{'max_depth': 6,
 'max_features': 19,
 'min_samples_leaf': 16,
 'min_weight_fraction_leaf': 0.0,
 'splitter': 'best'}

In [41]:
# fitting fine_tuned model for DecisonTreeRegressor using all available features and fine_tuned parameters
model_params = tuning_model.best_params_.copy()
model_params["criterion"] = "squared_error"

train_results_tree_fine, test_results_tree_fine, model_tree_fine = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressor",
                                                                              model_params)

In [42]:
# fitting fine_tuned model for RandomForestRegressor using all available features and fine_tuned parameters
model_params = tuning_model.best_params_.copy()
model_params["criterion"] = "squared_error"
del model_params["splitter"]
model_params["n_estimators"] = 100

train_results_rf_fine, test_results_rf_fine, model_rf_fine = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressorRandomForest",
                                                                              model_params)

In [43]:
train_result_list = [train_results_tree_base, train_results_tree_fine, train_results_rf_base, train_results_rf_fine]

test_result_list = [test_results_tree_base, test_results_tree_fine, test_results_rf_base, test_results_rf_fine]

model_names = ["Tree (Base)", "Tree (Fine-tuned)", "RF (Base)", "RF (Fine-tuned)"]

tab = tabularize_model_metrics(train_result_list, test_result_list, model_names)
round(tab, 3)

,Model,Train Mean Sq Error,Test Mean Sq Error,Train R2,Test R2,Train Adjusted R2,Test Adjusted R2
0,Tree (Base),0.093,207.091,1.000,-0.049,1.000,-0.054
1,Tree (Fine-tuned),102.333,106.451,0.495,0.461,0.494,0.458
2,RF (Base),15.186,108.860,0.925,0.449,0.925,0.446
3,RF (Fine-tuned),98.668,102.150,0.513,0.483,0.512,0.480


In [44]:
print(round(tab, 2).to_latex())

\begin{tabular}{llrrrrrr}
\toprule
 & Model & Train Mean Sq Error & Test Mean Sq Error & Train R2 & Test R2 & Train Adjusted R2 & Test Adjusted R2 \\
\midrule
0 & Tree (Base) & 0.090000 & 207.090000 & 1.000000 & -0.050000 & 1.000000 & -0.050000 \\
1 & Tree (Fine-tuned) & 102.330000 & 106.450000 & 0.490000 & 0.460000 & 0.490000 & 0.460000 \\
2 & RF (Base) & 15.190000 & 108.860000 & 0.930000 & 0.450000 & 0.920000 & 0.450000 \\
3 & RF (Fine-tuned) & 98.670000 & 102.150000 & 0.510000 & 0.480000 & 0.510000 & 0.480000 \\
\bottomrule
\end{tabular}

